# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import keras
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 48s 0us/step


In [3]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [4]:
import numpy as np
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [5]:
x_train_lt5 = x_train[y_train<5]
y_train_lt5 = y_train[y_train<5]
x_test_lt5 = x_test[y_test<5]
y_test_lt5 = y_test[y_test<5]

x_train_gt5 = x_train[y_train>=5]
y_train_gt5 = y_train[y_train>=5]
x_test_gt5 = x_test[y_test>=5]
y_test_gt5 = y_test[y_test>=5]

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [6]:
import pandas as pd
y_train_lt5 = pd.get_dummies(y_train_lt5)
y_test_lt5 = pd.get_dummies(y_test_lt5)

y_train_gt5 = pd.get_dummies(y_train_gt5)
y_test_gt5 = pd.get_dummies(y_test_gt5)

In [10]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)
print("=============")
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(25000, 32, 32, 3)
(25000, 5)
(5000, 32, 32, 3)
(5000, 5)
(25000, 32, 32, 3)
(25000, 5)
(5000, 32, 32, 3)
(5000, 5)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [11]:
from keras import Sequential
from keras.layers import Flatten,BatchNormalization,MaxPooling2D,Dropout, Convolution2D, Dense

In [13]:
model = Sequential()
model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Convolution2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
#model.add(BatchNormalization())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation = 'softmax'))

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 6, 64)         

In [15]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_lt5,y_train_lt5, validation_data=(x_test_lt5,y_test_lt5), epochs = 30, batch_size= 32) 

model.evaluate(x_test_lt5,y_test_lt5)

W1124 21:14:57.621248 22596 deprecation_wrapper.py:119] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1124 21:14:57.859333 22596 deprecation.py:323] From C:\Users\VKE\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 5000 samples
Epoch 1/30
25000/25000 [==============================] - 22s 889us/step - loss: 1.0102 - acc: 0.5968 - val_loss: 0.7835 - val_acc: 0.7066TA: 9s - loss: 1.0911 - acc: 0. - ETA: 8s - loss: 1.0884 - acc - ETA: 8s - loss: 1.0816 - acc: 0.562 - ETA: 8s - loss: 1.080 - ETA: 6s - loss: 1.0658 - a - ETA: 5s  - ETA: 3s - loss: 1.0335 - acc: - ETA: 2s - loss: - ETA: 0s - loss: 1.0175 
Epoch 2/30
25000/25000 [==============================] - 18s 724us/step - loss: 0.7787 - acc: 0.6965 - val_loss: 0.6807 - val_acc: 0.7378 loss: 0.8950 - acc - ETA: 15s - loss: 0.8991 - acc: 0.64 - ETA: 15s - loss: 0.8976 - ETA - ETA: 14s - loss: 0.8382 - ETA: 14s - loss: 0.8317 - ETA: 12s - loss: 0.82 - E - ETA: 10s  - ETA: 9s - lo - ETA: 8s - loss: 0.8038 - acc: 0.688 - ETA: 8s - loss: 0.8037 - a - ETA: 7s - los - ETA: 6s - loss: 0.7970 - - ETA: 5s - lo - ETA: 1s - loss: 0.7 - ETA: 0s - loss: 0.7793 - acc: 0.6
Epoch 3/30
25000/25000 [==============================

[0.5501628074645996, 0.8254]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [16]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False

batch_normalization_2
conv2d_3
max_pooling2d_3
dropout_4
conv2d_4
max_pooling2d_4
dropout_5
flatten_2
dense_3
dropout_6
dense_4


In [17]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

batch_normalization_2
False
conv2d_3
False
max_pooling2d_3
False
dropout_4
False
conv2d_4
False
max_pooling2d_4
False
dropout_5
False
flatten_2
False
dense_3
True
dropout_6
False
dense_4
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [18]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_gt5,y_train_gt5,validation_data=(x_test_gt5,y_test_gt5), epochs = 10, batch_size= 32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 11s 434us/step - loss: 1.2838 - acc: 0.5826 - val_loss: 0.6097 - val_acc: 0.7788
Epoch 2/10
25000/25000 [==============================] - 10s 416us/step - loss: 0.6788 - acc: 0.7541 - val_loss: 0.5156 - val_acc: 0.8164
Epoch 3/10
25000/25000 [==============================] - 11s 425us/step - loss: 0.5826 - acc: 0.7900 - val_loss: 0.4755 - val_acc: 0.8326
Epoch 4/10
25000/25000 [==============================] - 11s 438us/step - loss: 0.5213 - acc: 0.8114 - val_loss: 0.4334 - val_acc: 0.8480
Epoch 5/10
25000/25000 [==============================] - 11s 433us/step - loss: 0.4686 - acc: 0.8310 - val_loss: 0.4200 - val_acc: 0.8514
Epoch 6/10
25000/25000 [==============================] - 10s 418us/step - loss: 0.4382 - acc: 0.8412 - val_loss: 0.4056 - val_acc: 0.8608
Epoch 7/10
25000/25000 [==============================] - 11s 420us/step - loss: 0.4051 - acc: 0.8538 - val_loss: 0.3

In [19]:
model.evaluate(x_test_gt5,y_test_gt5)

5000/5000 [==============================] - 1s 215us/step


[0.3838233545780182, 0.8682]

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [21]:
import pandas as pd
data = pd.read_csv('./tweets.csv', encoding = "ISO-8859-1").dropna()

In [22]:
data.shape

(3291, 3)

In [23]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [27]:
data.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,3191,3191,3191
unique,3182,9,2
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,Positive emotion
freq,3,918,2672


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [24]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [25]:
data.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(data['tweet_text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [29]:
doc_matrix = cv.transform(data.tweet_text)

In [30]:
import numpy as np
doc_matrix1 = doc_matrix.toarray()

In [31]:
doc_matrix1.shape

(3191, 5648)

### 8. Find number of different words in vocabulary

In [32]:
print(doc_matrix)

  (0, 79)	1
  (0, 227)	1
  (0, 417)	2
  (0, 1290)	1
  (0, 2286)	1
  (0, 2426)	1
  (0, 2641)	1
  (0, 2663)	1
  (0, 3304)	1
  (0, 3706)	1
  (0, 4145)	1
  (0, 4619)	1
  (0, 4772)	1
  (0, 5014)	1
  (0, 5144)	1
  (0, 5230)	1
  (0, 5373)	1
  (0, 5416)	1
  (1, 152)	1
  (1, 281)	1
  (1, 347)	1
  (1, 367)	1
  (1, 417)	1
  (1, 475)	1
  (1, 1351)	1
  :	:
  (3189, 286)	1
  (3189, 302)	2
  (3189, 347)	1
  (3189, 798)	1
  (3189, 800)	1
  (3189, 1816)	1
  (3189, 1936)	2
  (3189, 2275)	1
  (3189, 2486)	1
  (3189, 2631)	1
  (3189, 2641)	1
  (3189, 2663)	1
  (3189, 3209)	1
  (3189, 3280)	1
  (3189, 4203)	1
  (3189, 4595)	1
  (3189, 4710)	1
  (3189, 4772)	1
  (3189, 4784)	1
  (3189, 5247)	1
  (3189, 5277)	1
  (3190, 1699)	1
  (3190, 2631)	1
  (3190, 2909)	1
  (3190, 4772)	1


#### Tip: To see all available functions for an Object use dir

In [33]:
dir(cv)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_sort_features',
 '_stop_words_id',
 '_validate_custom_analyzer',
 '_validate_params',
 '_validate_vocabulary',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'fit',
 'fit_transform',
 'fixed_vocabulary_',
 'get_feature_names',
 'get_params',
 'get_stop_words',
 'input',
 'inverse_transf

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [34]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [35]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [38]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
x = data['tweet_text']
y = data['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [40]:
vect = CountVectorizer()

# create document-term matrices
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

In [41]:
print(x_train_dtm.shape)
print(x_test_dtm.shape)

(2393, 4919)
(798, 4919)


In [42]:
print('Features: ', x_train_dtm.shape[1])

Features:  4919


In [43]:
nb = MultinomialNB()
nb.fit(x_train_dtm, y_train)
y_pred_class = nb.predict(x_test_dtm)

In [47]:
# calculate accuracy
print(accuracy_score(y_test, y_pred_class))

0.8471177944862155


In [50]:
# use logistic regression with text column only
logreg = LogisticRegression()
logreg.fit(x_train_dtm, y_train)
y_pred_class = logreg.predict(x_test_dtm)
print(accuracy_score(y_test, y_pred_class))

0.868421052631579


## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [51]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(x_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(x_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [52]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  24855
Accuracy:  0.8558897243107769


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [53]:
# remove English stop words
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  4681
Accuracy:  0.8533834586466166


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [54]:
# remove English stop words and only keep 100 features
vect = CountVectorizer(stop_words='english', max_features=300)
tokenize_test(vect)

Features:  300
Accuracy:  0.8107769423558897


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [55]:
# include 1-grams and 2-grams, and limit the number of features
vect = CountVectorizer(ngram_range=(1, 2), max_features=15000)
tokenize_test(vect)

Features:  15000
Accuracy:  0.8533834586466166


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [56]:
# include 1-grams and 2-grams, and only include terms that appear at least 2 times
vect = CountVectorizer(ngram_range=(1, 2), min_df=2)
tokenize_test(vect)

Features:  7764
Accuracy:  0.8583959899749374
